In [39]:
module lc

import Base: unsafe_convert, write

const liblcm = "/home/rdeits/.julia/v0.5/PyLCM/deps/usr/lib/liblcm.so"

type LCM
    pointer::Ptr{Void}

    LCM() = begin
        lc = new(ccall((:lcm_create, liblcm), Ptr{Void}, (Ptr{UInt8},), ""))
        finalizer(lc, close)
        lc
    end
end
unsafe_convert(::Type{Ptr{Void}}, lc::LCM) = lc.pointer

function close(lcm::LCM)
	if lcm.pointer != C_NULL
	    ccall((:lcm_destroy, liblcm), Void, (Ptr{Void},), lcm)
		lcm.pointer = C_NULL
	end
end

function publish(lcm::LCM, channel::String, data::Vector{UInt8})
    ccall((:lcm_publish, liblcm), Cint, (Ptr{LCM}, Ptr{UInt8}, Ptr{UInt8}, Cuint), channel, data, length(data))
end


immutable CommsT
    utime::Int64
    format::String
    format_version_major::Int32
    format_version_minor::Int32
    data::Vector{UInt8}
end

function write(io::IO, comms::CommsT)
    const signature = UInt8[211, 104, 224, 63, 51, 197, 104, 190]
    write(io, signature)
    write(io, hton(comms.utime))
    write(io, hton(UInt32(length(comms.format) + 1)))
    write(io, comms.format)
    write(io, UInt8(0))
    write(io, hton(comms.format_version_major))
    write(io, hton(comms.format_version_minor))
    write(io, hton(Int32(length(comms.data))))
    write(io, hton.(comms.data))
end

end

lc

In [40]:
b = IOBuffer()

IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1)

In [41]:
data = "{\"hello\": \"world\"}"
msg = lc.CommsT(123456, "my format", 15, 3, data)

lc.CommsT(123456,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])

In [42]:
write(b, msg)

0x0000000000000012

In [43]:
seekstart(b)
encoded = Int.(readbytes(b))
println(encoded)

[211,104,224,63,51,197,104,190,0,0,0,0,0,1,226,64,0,0,0,10,109,121,32,102,111,114,109,97,116,0,0,0,0,15,0,0,0,3,0,0,0,18,123,34,104,101,108,108,111,34,58,32,34,119,111,114,108,100,34,125]


In [44]:
encoded == expected

true

In [37]:
expected = [211,
 104,
 224,
 63,
 51,
 197,
 104,
 190,
 0,
 0,
 0,
 0,
 0,
 1,
 226,
 64,
 0,
 0,
 0,
 10,
 109,
 121,
 32,
 102,
 111,
 114,
 109,
 97,
 116,
 0,
 0,
 0,
 0,
 15,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 18,
 123,
 34,
 104,
 101,
 108,
 108,
 111,
 34,
 58,
 32,
 34,
 119,
 111,
 114,
 108,
 100,
 34,
 125]


60-element Array{Int64,1}:
 211
 104
 224
  63
  51
 197
 104
 190
   0
   0
   0
   0
   0
   ⋮
 111
  34
  58
  32
  34
 119
 111
 114
 108
 100
  34
 125

In [16]:
hton(Int64(1))

72057594037927936

In [3]:
sig = r"\xd3h\xe0?3\xc5h\xbe"

r"\xd3h\xe0?3\xc5h\xbe"